## Sample execution of the repository

The config file must be set accordingly. The default directs to the data folder, where the minimal example will be downloaded to and other files generated as well. This can be easiyl set to an external location if the data become large.

Then, a minimal working example file can be downloaded from (TODO) for December (12) of 2009 for the GOCE satellite mission to showcase the use of the repository.

In [1]:
import urllib.request
import os
from pathlib import Path

def download_file(url, dir, filename):
    if not os.path.exists(dir):
        os.makedirs(dir)
    print(f"Downloading {filename}...")
    filename = dir + filename
    urllib.request.urlretrieve(url, filename)
    print(f"Downloading {filename} completed")

dirname = os.path.abspath('')
print("prev dirname: ", dirname)
orbit_path = os.path.join(dirname, 'data/orbit/')
goce_path = os.path.join(dirname, 'data/raw/')
write_path = os.path.join(dirname, 'data/intermediate/')
download_file('ftp://magftp.gfz-potsdam.de/PyPlatMag/GO_data_200912.h5', goce_path, 'GO_data_200912.h5')
download_file('ftp://magftp.gfz-potsdam.de/PyPlatMag/GO_rawdata_200912.h5', goce_path, 'GO_rawdata_200912.h5')
download_file('ftp://magftp.gfz-potsdam.de/PyPlatMag/GO_rawdata_hk_200912.h5', goce_path, 'GO_rawdata_hk_200912.h5')
download_file('ftp://magftp.gfz-potsdam.de/PyPlatMag/GO_rawdata_telemetry_200912.h5', goce_path, 'GO_rawdata_telemetry_200912.h5')
download_file('ftp://magftp.gfz-potsdam.de/PyPlatMag/GO_ORBCNT_20090317T231001_20131109T234516_0101.h5', orbit_path, 'GO_ORBCNT_20090317T231001_20131109T234516_0101.h5')


prev dirname:  /home/michaeli/prj/MAG/PlatformMagnetometers/pyplatmag


abc

In [2]:
import ruamel.yaml
yaml = ruamel.yaml.YAML()
with open('config.yaml') as fp:
    data = yaml.load(fp)
    data['goce_data_path'] = goce_path
    data['write_path'] = write_path
    data['orbit_counter_path'] = orbit_path + 'GO_ORBCNT_20090317T231001_20131109T234516_0101.h5'
    ret = ruamel.yaml.comments.CommentedSeq()
    ret.fa.set_flow_style()
    ret.append('200912')
    data['year_month_specifiers'] = ret

with open('config.yaml', 'wb') as fp:
    yaml.dump(data, fp)

with open('config_goce.yaml') as fp:
    data = yaml.load(fp)
    data['cdf_config']['cdf_path'] = 'data/cdf/'
    data['train_config']['learn_config']['epochs'] = 20
    data['train_config']['learn_config']['epochs_finetune'] = 20


with open('config_goce.yaml', 'wb') as fp:
    yaml.dump(data, fp)

abc

In [3]:
from scripts.read_files import read_files

read_files()

/home/michaeli/prj/MAG/PlatformMagnetometers/pyplatmag/scripts/read_files.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['RAW_Timestamp'] = th.gps_to_datetime(data['gps_sec'])
/home/michaeli/prj/MAG/PlatformMagnetometers/pyplatmag/preprocessing/space_weather_preprocessor.py:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["Dst"] = dst_interpolater(data["gps_sec"])
/home/michaeli/prj/MAG/PlatformMagnetometers/pyplatmag/preprocessing/space_weather_preprocessor.py:76: PerformanceWarning: DataFrame is highly fragmen

abc

In [4]:
from scripts.read_files_nan_handling import read_files_nan_handling

read_files_nan_handling()

abc

In [5]:
from scripts.train_pinn import train_pinn

train_pinn()

2024-10-28 15:39:50.788857: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


writing x_all_columns_file.


2024-10-28 15:40:10.955649: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-28 15:40:10.987289: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-28 15:40:10.987565: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 515)]                0         []                            
                                                                                                  
 dense (Dense)               (None, 384)                  198144    ['input_1[0][0]']             
                                                                                                  
 inputs_current_0 (InputLay  [(1500, 1)]                  0         []                            
 er)                                                                                              
                                                                                                  
 inputs_current_1 (InputLay  [(1500, 1)]                  0         []                        

2024-10-28 15:40:49.399237: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f95127f7c30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-28 15:40:49.399266: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-10-28 15:40:49.548965: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-28 15:40:49.795781: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8907
2024-10-28 15:40:50.343755: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


76/76 [==============================] - 66s 138ms/step - loss: 175728880.0000 - mse: 246522576.0000 - mae: 10594.7891 - val_loss: 38369096.0000 - val_mse: 45977384.0000 - val_mae: 5225.3569 - lr: 0.0100
Epoch 2/20
76/76 [==============================] - 6s 83ms/step - loss: 30885410.0000 - mse: 39347736.0000 - mae: 4762.2539 - val_loss: 24614478.0000 - val_mse: 33496506.0000 - val_mae: 4414.0757 - lr: 0.0100
Epoch 3/20
76/76 [==============================] - 6s 82ms/step - loss: 18964236.0000 - mse: 29826684.0000 - mae: 4138.0195 - val_loss: 14118599.0000 - val_mse: 24844280.0000 - val_mae: 3783.9214 - lr: 0.0100
Epoch 4/20
76/76 [==============================] - 6s 82ms/step - loss: 10673206.0000 - mse: 20972628.0000 - mae: 3398.7581 - val_loss: 6849085.5000 - val_mse: 16220178.0000 - val_mae: 2859.4792 - lr: 0.0100
Epoch 5/20
76/76 [==============================] - 6s 82ms/step - loss: 4909427.5000 - mse: 12692508.0000 - mae: 2389.9929 - val_loss: 3843928.5000 - val_mse: 1095797

/home/michaeli/anaconda3/envs/pyplatmag/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


19/19 [==============================] - 1s 26ms/step


abc

In [6]:
from scripts.train_pinn_finetune import train_pinn_finetune

train_pinn_finetune()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 515)]                0         []                            
                                                                                                  
 dense_187 (Dense)           (None, 384)                  198144    ['input_2[0][0]']             
                                                                                                  
 inputs_current_0 (InputLay  [(1500, 1)]                  0         []                            
 er)                                                                                              
                                                                                                  
 inputs_current_1 (InputLay  [(1500, 1)]                  0         []                      

/home/michaeli/anaconda3/envs/pyplatmag/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


19/19 [==============================] - 1s 27ms/step


In [7]:
from scripts.generate_cdf_files import generate_cdf_files

generate_cdf_files()

/home/michaeli/.local/lib/python3.8/site-packages/spacepy/time.py:2367: UserWarning: Leapseconds may be out of date. Use spacepy.toolbox.update(leapsecs=True)
  warnings.warn('Leapseconds may be out of date.'


2024-10-28 15:47:09,453 [WARNING] data_filters.goce_filter: WARNING: No Magnetic-Activity-Filtering, but Flag added.
2024-10-28 15:47:12,910 [WARNING] data_filters.goce_filter: WARNING: No Spaceweather Flag removal, but Flag added.
2024-10-28 15:47:12,918 [WARNING] data_filters.goce_filter: WARNING: No outlier removal, but Flag added.
2024-10-28 15:47:12,920 [WARNING] data_filters.goce_filter: WARNING: No interpolation distance filtering, but Flag added.
model_input.shape: (154157, 698)
model_input.columns: ['MHT00000', 'PHD94001', 'PHD94002', 'PHD94003', 'PHD94004', 'PHD94005', 'PHD94006', 'PHD95002', 'PHD95021', 'THT00004', 'THT00012', 'THT00068', 'THT10000', 'THT10001', 'THT10002', 'THT10003', 'THT10004', 'THT10005', 'FGM1_X', 'FGM1_Y', 'FGM1_Z', 'FGM2_X', 'FGM2_Y', 'FGM2_Z', 'FGM3_X', 'FGM3_Y', 'fgm_gapsize', 'mtr1_current', 'mtr2_current', 'mtr3_current', 'ACCGRID_CURRENT_1_1i', 'ACCGRID_CURRENT_8_1i', 'ACCGRID_TRIP_CNT_8_1i', 'ACCGRID_VOLTAGE_8_1i', 'ANODE_CURRENT_8_1i', 'ANODE_V